# 計算物理III 数値磁性体物性入門

## 6.2.3 部分ボルツマン因子の計算

1次元XXXハイゼンベルグモデルのn次近似分配関数を求める。

ハミルトニアン
$$
\mathcal{H}
= \sum_{j=1}^{L} \mathcal{H}_{j,j+1}
= \sum_{j} \left\{
   -2J_{xy}\left(S_j^x S_{j+1}^x + S_j^y S_{j+1}^y \right)
   -2J_{z} S_j^z S_{j+1}^z
  \right\}.
$$

$L=4, n=2, \beta=1, J=1$でチェッカーボード分解してZnを求めるコード。電子配置は$2^{16}=65535$通りあり、それら全てのパターンに対して、部分ボルツマン因子の行列から成分を取り出して各黒マスごとの積を計算し、和を取って計算した。

おそらくもっと効率の良い計算方法（量子転送行列法？）があるはずだが、Theoryに従ってそのまま計算してみた。

結果は、90パターンの電子配置について、積が0以外になり、合計は$Z_n=44.627744732937494$となった。ChatGPTが出した解の一つと一致しているが確信が持てない...

（追記）
黒マスは許容配置、白マスが不許容配置を含んで90パターン。全マスが許容配置になるのは30パターンのみ。どちらで計算すべきかわからない。。

In [1]:
from re import A
import numpy as np
import math

J = 1
beta = 1
L = 4
n = 2

a_in_rho = math.exp(beta*J/(2*n))
b_in_rho = math.exp(-beta*J/(2*n))*math.cosh(beta*J/n)
c_in_rho = math.exp(-beta*J/(2*n))*math.sinh(beta*J/n)

def all_4x4_binary_matrices_array():
  """
  0..65535 を2進展開して 4x4 に並べた全パターンを
  mats.shape == (65536, 4, 4) の uint8 配列で返す。
  """
  N = 1 << L**2  # 65536
  n_ = np.arange(N, dtype=np.uint16)               # (65536,)
  bits = ((n_[:, None] >> np.arange(16, dtype=np.uint16)) & 1).astype(np.uint8)  # (65536,16)
  mats = bits.reshape(N, 4, 4)                    # (65536,4,4)
  return mats

def add_first_row_col(a: np.ndarray) -> np.ndarray:
    if a.shape != (4,4):
        raise ValueError("入力は4x4配列にしてください。")
    # 右に1列目を追加 → (4,5)
    b = np.hstack([a, a[:, [0]]])
    # 下に1行目（※いまの b の1行目）を追加 → (5,5)
    c = np.vstack([b, b[[0], :]])
    return c

def construct_indices_to_get_checkerboard_block(n_rows):
  indices = []
  for n in range(n_rows):
    if n % 2 == 0:
      indices.append((n, 0))
      indices.append((n, 2))
    else:
      indices.append((n, 1))
      indices.append((n, 3))
  return np.array(indices)

def get_value_from_rho(spin_latice):
  if np.array_equal(spin_latice, np.array([[0, 0],[0, 0]])):
    return a_in_rho
  elif np.array_equal(spin_latice, np.array([[1, 0],[1, 0]])):
    return b_in_rho
  elif np.array_equal(spin_latice, np.array([[0, 1],[0, 1]])):
    return b_in_rho
  elif np.array_equal(spin_latice, np.array([[1, 0],[0, 1]])):
    return c_in_rho
  elif np.array_equal(spin_latice, np.array([[0, 1],[1, 0]])):
    return c_in_rho
  elif np.array_equal(spin_latice, np.array([[1, 1],[1, 1]])):
    return a_in_rho
  else:
    return 0

def product_of_partial_boltzmann_factors(spin_blocks):
  product = 1
  for block in spin_blocks:
    val = get_value_from_rho(block)
    if val == 0:
      return 0
    product *= val

  return product


mats = all_4x4_binary_matrices_array()
indices = construct_indices_to_get_checkerboard_block(mats[0].shape[0])

# m = np.array([
#   [1, 0, 1, 0],
#   [0, 1, 0, 1],
#   [1, 0, 1, 0],
#   [0, 1, 0, 1],
# ])
# mats = np.array([m])
n_non_zero = 0
sum = 0

for mat in mats:
  # 周期的境界条件のため、行の末尾に最初の行を、列の末尾に最初の列を追加
  mat = add_first_row_col(mat)

  # チェッカーボード分解したスピン行列から、2x2のブロックを全通り取り出す
  blocks = np.lib.stride_tricks.sliding_window_view(mat, (2, 2))
  # 部分ボルツマン因子を取り出すブロックだけ取る
  r, c = np.array(indices).T
  picked = blocks[r, c]

  # 対象の部分ボルツマン因子の積を計算する
  product = product_of_partial_boltzmann_factors(picked)

  if product != 0:
    n_non_zero += 1
    sum += product

# n=2, L=4の場合、90通りの電子配列で0ではなければOK
print(n_non_zero)
print('Zn=', sum)

90
Zn= 44.627744732937494
